
polygraphy를 설치하기 위해서는 다음의 명령어를 사용해야한다.

`python -m pip install colored polygraphy --extra-index-url https://pypi.ngc.nvidia.com`

polygraphy는 의존하는 패키지가 많이 없기때문에 아래에 `AUTOINSTALL_DEPS`를 True로 바꿔주면 알아서 설치가된다.

In [1]:
import polygraphy
polygraphy.config.AUTOINSTALL_DEPS = True
polygraphy.config.ASK_BEFORE_INSTALL = True # 지 멋대로 설치되는 것을 방지하기 위해서
print(f"{polygraphy.config.AUTOINSTALL_DEPS=}")

polygraphy.config.AUTOINSTALL_DEPS=True


# Backends

`Bankends`는 deep learning framework와의 interface를 제공한다. Backends는 Loader와 Runner로 구성되어있다.

일단 Loader에 대해서 먼저 알아보자.
Polygraphy는 두 가지 Loader를 제공한다. `PascalCase`의 경우 TensorRT엔진을 만들 수 있는 callable을 반환하고, `snakecase`의 경우 callable이 아니라 engine을 반환한다.
즉 전자를 사용하면 나중에 호출할 때 engine이 생성되고, 후자를 사용하면 바로 engine이 생성된다.

In [2]:
from polygraphy.backend.trt import EngineFromNetwork, NetworkFromOnnxPath

build_engine = EngineFromNetwork(NetworkFromOnnxPath("/home/park/ros2/tensorRT/policy.onnx"))
print(type(build_engine))

[I] TF32 is disabled by default. Turn on TF32 for better performance with minor accuracy differences.
<class 'polygraphy.backend.trt.loader.EngineFromNetwork'>


In [3]:
from polygraphy.backend.trt import engine_from_network, network_from_onnx_path

engine = engine_from_network(network_from_onnx_path("/home/park/ros2/tensorRT/policy.onnx"))
print(type(engine))

[I] TF32 is disabled by default. Turn on TF32 for better performance with minor accuracy differences.
[I] Configuring with profiles:[
        Profile 0:
            {qpos [min=[1, 6], opt=[1, 6], max=[1, 6]],
             image [min=[1, 3, 3, 480, 640], opt=[1, 3, 3, 480, 640], max=[1, 3, 3, 480, 640]]}
    ]
[W] profileSharing0806 is on by default in TensorRT 10.0. This flag is deprecated and has no effect.
[I] Building engine with configuration:
    Flags                  | []
    Engine Capability      | EngineCapability.STANDARD
    Memory Pools           | [WORKSPACE: 11783.31 MiB, TACTIC_DRAM: 11783.31 MiB, TACTIC_SHARED_MEMORY: 1024.00 MiB]
    Tactic Sources         | [EDGE_MASK_CONVOLUTIONS, JIT_CONVOLUTIONS]
    Profiling Verbosity    | ProfilingVerbosity.DETAILED
    Preview Features       | [PROFILE_SHARING_0806]
[I] Finished engine building in 14.640 seconds
<class 'tensorrt.tensorrt.ICudaEngine'>


# Runner

Runner는 Loader를 사용해 모델을 로드하고, 추론을 실행하는 객체이다.
Runner를 사용하기 위해서는 activate를 해야하는데 한 번 activate하는데 비용이 크므로, 여러번 하지 않는 것이 좋다. 그리고 Context Manager를 사용하는 것을 권장한다.

In [4]:
from polygraphy.backend.trt import TrtRunner

with TrtRunner(build_engine) as runner:
    outputs = runner.infer(feed_dict={"input0": input_data})

[I] Configuring with profiles:[
        Profile 0:
            {qpos [min=[1, 6], opt=[1, 6], max=[1, 6]],
             image [min=[1, 3, 3, 480, 640], opt=[1, 3, 3, 480, 640], max=[1, 3, 3, 480, 640]]}
    ]
[I] Building engine with configuration:
    Flags                  | []
    Engine Capability      | EngineCapability.STANDARD
    Memory Pools           | [WORKSPACE: 11783.31 MiB, TACTIC_DRAM: 11783.31 MiB, TACTIC_SHARED_MEMORY: 1024.00 MiB]
    Tactic Sources         | [EDGE_MASK_CONVOLUTIONS, JIT_CONVOLUTIONS]
    Profiling Verbosity    | ProfilingVerbosity.DETAILED
    Preview Features       | [PROFILE_SHARING_0806]
[I] Finished engine building in 14.286 seconds


NameError: name 'input_data' is not defined

## ONNX Model Export

ONNX Model을 Export하기 위해서는 `torch.onnx.export`함수를 사용해야한다. 이 함수를 사용할 때 dummy input을 넣어줘야하는데 ACT신경망의 경우, 학습과 추론시에 신경망이 다르다. 따라서 추론시에 어떤 신경망을 사용하는지 살펴보고, 어떻게 dummy input을 구성해야하는지 알아보자. 아래 코드에서 policy가 ACT신경망을 말하는데 여기에 입력이 qpos와 curr_image임을 알 수 있다. 아래 코드는 ACT에서 사용된 ACTPolicy이다. `__call__`이 호출되었을 때 actions가 있으면 training을 진행하고, 없으면 Inference만 진행한다.

`onnx_transformation.py`에 관련 코드가 있습니다.

# Calibrator

Calibrator는 FP32값을 INT8로 양자화할 때 필요한 Scale과 Zero-point값을 계산해주는 클래스이다. 
Static Calibration의 경우 미리 그 값을 계산하기때문에 전체 학습데이터를 대표하는 `representative dataset`이 필요하다. Calibrator는 데이터를 받아서 신경망을 통과시킨 뒤 신경망의 각 노드들에서 scale과 zero-point를 계산한다. 따라서, ONNX model을 생성할 때 입력값으로 선정한 크기의 데이터를 생성하는 제너레이터를 인자로 넘겨주어야한다.

tensorRT에 소개된 예제 ResNet-50같은 경우는 전처리가 크게 필요하지않지만, ACT의 경우 전처리가 꽤 복잡하다.

`act/utils.py`에 `EpisodicDataset`을 참고하면 x가지 전처리를 확인할 수 있다. 우리는 입력으로 qpos와 image만을 전달하기때문에 이들의 전처리만 고려하면된다.
1. `act/utils.py`에 `get_norm_stats`함수를 사용해 h5py로 표현된 전체 데이터셋의 qpos의 평균과 분산을 계산한다. 그리고 이 값을 통해 qpos를 정규화한다.
2. h5py에 저장된 이미지들을 하나의 numpy array로 합치고, (h w c)로 표현된 이미지를 (c h w)형태로 변환해준다.
3. 이미지 데이터를 255로 나눠 정규화한다

# IBuilderConfig

아래와 같이 `IBuilderConfig`를 작성한다. INT8 양자화를 지원하지않는 레어이가 존재하면 FP32로 계산된다. 이때, FP32가 아닌 FP16으로 계산을 원한다면 `fp16=True` 을 설정해주면 된다.

In [ ]:
# Each type flag must be set to true.
builder_config = poly_trt.create_config(builder=builder,
                                        network=network,
                                        int8=True,
                                        fp16=True,
                                        calibrator=calibrator)

# TensorRT Engine Build and Save

In [1]:
engine = poly_trt.engine_from_network(network=(builder, network, parser),
                                      config=builder_config)

# TensorRT engine will be saved to ENGINE_PATH.
ENGINE_PATH = "/home/park/ros2/tensorRT/act_int8_fp16.engine"
poly_trt.save_engine(engine, ENGINE_PATH)

# Load serialized engine using 'open'.
engine = poly_trt.engine_from_bytes(open(ENGINE_PATH, "rb").read())
print("engine이 저장되었습니다.")

NameError: name 'poly_trt' is not defined

이렇게 Engine을 생성하면 아래와 같은 경고문구가 발생합니다. engine은 성공적으로 생성되긴합니다.
아래 경고문구들은 tensorRT가 특정 레이어에서 Scale과 zero-point를 계산하지 못해 int8연산을 하지못하므로 fp16이나 fp32로 게산을 수행하겠다는 경고입니다.

저 같은 경우 다음과 같은 Warning이 발생했습니다.
[W] Missing scale and zero-point for tensor ONNXTRT_Broadcast_1182_output, expect fall back to non-int8 implementation for any layer consuming or producing given tensor
[W] Missing scale and zero-point for tensor model/backbones.0/backbones.0.1/Constant_1_output_0_output, expect fall back to non-int8 implementation for any layer consuming or producing given tensor
[W] Missing scale and zero-point for tensor model.transformer.encoder.layers.3.norm1.weight_output, expect fall back to non-int8 implementation for any layer consuming or producing given tensor
[W] Missing scale and zero-point for tensor model.transformer.decoder.layers.0.norm2.weight_output, expect fall back to non-int8 implementation for any layer consuming or producing given tensor

어디서 scale과 zero-point를 계산못하는지를 살펴보면 아ㄹ의 레이어들에서 못합니다.

1. model/backbones.0/backbones.0.1/Constant_1_output_0_output
2. model.transformer.encoder.layers.3.norm1.weight_output
3. model.transformer.decoder.layers.0.norm2.weight_output
4. ONNXTRT_Broadcast_1182

netron을 통해서 살펴보면 첫 번째는 position encoding과정에서 레이어이고, 두 번째, 세 번째는 Transformer에 layer normalization입니다. 네 번째는 잘 모르겠습니다.
github에 찾아보니 layer normalization의 경우 원래 안되는 거라고해서 일단은 넘어갔습니다.
[Q-DQ nodes for int8 LayerNorm #4084](https://github.com/NVIDIA/TensorRT/issues/4084)

# Engine 성능 비교

Engine의 성능을 비교하기 위해 다음 네 가지 타입의 Engine을 생성합니다.
1. act pytorch model
2. tensorRT fp32 origin model
3. INT8 Static Calibration 적용된 엔진
4. INT8 Random Calibration 적용된 엔진(Calibration으로 인한 정확도 향상을 알아보기위해)

위 engine들의 성능 비교 코드는 `compare.py`에 있습니다.
성능 비교 결과는 아래와 같습니다.
inference시에는 KL Divergence 에러를 계산할 수 없으므로 L1 Loss만을 사용해서 정확도를 평가하였습니다.
평가데이터는 총 40개를 사용하였습니다.

act_pytorch에서 tensorRT엔진을 만드는 것만으로도 추론시간이 약 2배 빨라졌고, INT8 양자화를 통해 L1 Loss가 약간 올라갔으나 약 7배 향상하였습니다.